In [1]:
%autosave 300
%load_ext autoreload
%autoreload 2
%reload_ext autoreload
%config Completer.use_jedi = False

Autosaving every 300 seconds


In [2]:
import os

os.chdir("../")
print(os.getcwd())

c:\workspace\EagSession1\eag_agentic_rag\url_rag


In [3]:
from IPython.display import display, HTML, Markdown
from dotenv import load_dotenv

In [4]:
# Load environment variables
load_dotenv()

# Clear SSL_CERT_FILE environment variable if set
if "SSL_CERT_FILE" in os.environ:
    del os.environ["SSL_CERT_FILE"]

In [5]:
from client.llm_provider import default_llm
from client.embedding_provider import OpenAIEmbeddingProvider

In [6]:
llm = default_llm.chat_model
embedder = OpenAIEmbeddingProvider().embeddings

In [7]:
print(llm.invoke("what is the capital of France?"))
print(len(embedder.embed_query("what is the capital of France?")))

content='The capital of France is Paris.' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 8, 'prompt_tokens': 14, 'total_tokens': 22, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_d8864f8b6b', 'id': 'chatcmpl-BTuiHUODFWzul7gHbJCtfJl8t2LS7', 'finish_reason': 'stop', 'logprobs': None} id='run-155e42e9-6360-42bc-8353-5cdb6aacfb85-0' usage_metadata={'input_tokens': 14, 'output_tokens': 8, 'total_tokens': 22, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}
1536


#### Embedding Generation Steps
- A function to which I will provide a url and it will scrape the content of the webpage
- We will token chunk the content of the webpage
- We will generate embeddings for each chunk
- We will store the embeddings in a vector database


In [8]:
# website = "https://geshan.com.np/blog/2018/11/4-ways-docker-changed-the-way-software-engineers-work-in-past-half-decade"

In [28]:
website = "https://www.redhat.com/en/topics/devops/what-is-helm"

In [29]:
from langchain_community.document_loaders import AsyncHtmlLoader
from langchain_community.document_transformers import Html2TextTransformer

In [30]:
# loader = AsyncHtmlLoader([website])
# docs = loader.load()

# html2text = Html2TextTransformer()
# docs_transformed = html2text.transform_documents(docs)

In [31]:
# docs_transformed[0].metadata

In [32]:
# Markdown(docs_transformed[0].page_content)

In [33]:
from langchain_text_splitters import TokenTextSplitter

In [34]:
# text_splitter = TokenTextSplitter(chunk_size=500, chunk_overlap=50,  model_name="gpt-4o",)

# texts_chunked = text_splitter.split_documents(docs_transformed)

In [35]:
# len(texts_chunked)

In [36]:
# texts_chunked[0].metadata

In [37]:
def extract_text_from_url(url):
    """
    Extract and return plain text Document(s) from a given URL using LangChain's AsyncHtmlLoader and Html2TextTransformer.
    Returns a list of Document objects.
    """
    loader = AsyncHtmlLoader([url])
    docs = loader.load()
    html2text = Html2TextTransformer()
    docs_transformed = html2text.transform_documents(docs)
    return docs_transformed


def chunk_text_documents(
    docs_transformed, chunk_size=500, chunk_overlap=50, model_name="gpt-4o"
):
    """
    Split the transformed documents into chunks using TokenTextSplitter.
    Returns a list of chunked Document objects.
    """
    text_splitter = TokenTextSplitter(
        chunk_size=chunk_size, chunk_overlap=chunk_overlap, model_name=model_name
    )
    texts_chunked = text_splitter.split_documents(docs_transformed)
    print(
        f"Number of chunked documents: {len(texts_chunked)} extracted from {len(docs_transformed)} original documents."
    )
    return texts_chunked

In [38]:
raw_docs = extract_text_from_url(website)
chunked_docs = chunk_text_documents(
    raw_docs, chunk_size=500, chunk_overlap=50, model_name="gpt-4o"
)

Fetching pages: 100%|##########| 1/1 [00:00<00:00,  6.89it/s]


Number of chunked documents: 9 extracted from 1 original documents.


In [39]:
import os
import uuid
from langchain_community.vectorstores import FAISS
from langchain_community.docstore.in_memory import InMemoryDocstore
import faiss

In [40]:
def add_documents_to_faiss_index(documents, embedder, index_name):
    """
    Adds documents to a FAISS vector store with the given index name.
    If the index does not exist, it creates a new one.
    If it exists, it loads and updates it.
    Each document is assigned a new UUID as its ID.
    """
    # Ensure the index folder exists or not
    index_name = f"./{index_name}"
    if not os.path.exists(index_name):
        print(f"Index folder '{index_name}' does not exist. Creating a new index.")
        # Create new FAISS index
        # Get embedding size from a sample embedding
        sample_embedding = embedder.embed_query("sample text")
        index = faiss.IndexFlatL2(len(sample_embedding))
        vector_store = FAISS(
            embedding_function=embedder,
            index=index,
            docstore=InMemoryDocstore(),
            index_to_docstore_id={},
        )
        print(f"Creating new FAISS index at '{index_name}'")
    else:
        print(f"Index folder '{index_name}' exists. Loading existing index.")
        # Load existing FAISS index
        vector_store = FAISS.load_local(
            index_name, embedder, allow_dangerous_deserialization=True
        )
        print(f"Loaded existing FAISS index from '{index_name}'")

    # Generate unique IDs for each document
    doc_ids = [str(uuid.uuid4()) for _ in documents]
    # Add documents to the vector store
    vector_store.add_documents(documents=documents, ids=doc_ids)
    print(f"Added {len(documents)} documents to the vector store.")

    # Save the updated index
    vector_store.save_local(folder_path=index_name)
    print(f"Index saved at '{index_name}'")

    return doc_ids  # Optionally return the IDs for reference

In [41]:
index_name = "test_index"
ids = add_documents_to_faiss_index(chunked_docs, embedder, index_name)

Index folder './test_index' exists. Loading existing index.
Loaded existing FAISS index from './test_index'
Added 9 documents to the vector store.
Index saved at './test_index'


In [42]:
# read the pickle file
path = "./test_index/index.pkl"
import pickle

with open(path, "rb") as f:
    data = pickle.load(f)

In [44]:
def get_vector_store(index_name, embedder):
    """
    Load a FAISS vector store from the specified index name.
    """
    path = f"./{index_name}"
    if not os.path.exists(path):
        print(f"Index folder '{index_name}' does not exist. Creating a new index.")
        raise FileNotFoundError(f"Index folder '{index_name}' does not exist.")
    # Load existing FAISS index
    vector_store = FAISS.load_local(
        path, embedder, allow_dangerous_deserialization=True
    )
    print(f"Loaded existing FAISS index from '{index_name}'")
    return vector_store

In [45]:
vector_store = get_vector_store(index_name, embedder)

Loaded existing FAISS index from 'test_index'


In [46]:
vector_store.similarity_search(
    query="What is Helm?",
    k=3,
)

[Document(id='dcf74748-0e94-4bf5-b563-35a5bf9fa5b0', metadata={'source': 'https://www.redhat.com/en/topics/devops/what-is-helm', 'title': 'What is Helm?', 'description': 'Helm is a package manager for Kubernetes that includes all the necessary code and resources needed to deploy an application to a cluster.', 'language': 'en'}, page_content='日本語한국어PortuguêsEspañol\n\nContact us\n\nEnglish\n\n### Select a language\n\n  * 简体中文\n  * English\n  * Français\n  * Deutsch\n  * Italiano\n  * 日本語\n  * 한국어\n  * Português\n  * Español\n\nSelect a language简体中文EnglishFrançaisDeutschItaliano日本語한국어PortuguêsEspañol\n\nRed Hat\n\n  * AI\n  * Hybrid cloud\n  * Products\n  * Training & services\n  * Resources\n  * Partners\n  * About\n\nMenu Search  For you  Contact us  English  Log In\n\n  * AI\n  * Hybrid cloud\n  * Products\n  * Training & services\n  * Resources\n  * Partners\n  * About\n  * Contact us\n\n### Select a language\n\n  * 简体中文\n  * English\n  * Français\n  * Deutsch\n  * Italiano\n  * 日本語\

In [48]:
from langchain_core.documents import Document

In [49]:
class FaissConversationStore:
    """
    Store and retrieve conversations in a FAISS vector store using conversation IDs.
    Each message is a Document with metadata: conversation_id, sender ('human' or 'ai'), and order.
    """

    def __init__(self, embedder, index_folder="history_index"):
        self.embedder = embedder
        self.index_folder = index_folder
        if not os.path.exists(index_folder):
            sample_embedding = embedder.embed_query("sample text")
            index = faiss.IndexFlatL2(len(sample_embedding))
            self.vector_store = FAISS(
                embedding_function=embedder,
                index=index,
                docstore=InMemoryDocstore(),
                index_to_docstore_id={},
            )
            self.vector_store.save_local(folder_path=index_folder)
        else:
            self.vector_store = FAISS.load_local(
                index_folder, embedder, allow_dangerous_deserialization=True
            )

    def store_conversation(self, conversation_id: str, messages: list[dict]):
        """
        Store a conversation as a list of messages.
        Each message is a dict: {'sender': 'human'/'ai', 'content': str}
        """
        docs = [
            Document(
                page_content=msg["content"],
                metadata={
                    "conversation_id": conversation_id,
                    "sender": msg["sender"],
                    "order": i,
                },
            )
            for i, msg in enumerate(messages)
        ]
        ids = [f"{conversation_id}_{i}" for i in range(len(messages))]
        self.vector_store.add_documents(docs, ids=ids)
        self.vector_store.save_local(folder_path=self.index_folder)

    def get_conversation(self, conversation_id: str) -> list[dict]:
        """
        Retrieve the conversation as a list of dicts with sender and content, ordered.
        """
        # Get all docstore keys for this conversation
        ids = [
            k
            for k in self.vector_store.docstore._dict.keys()
            if k.startswith(conversation_id + "_")
        ]
        docs = self.vector_store.get_by_ids(ids)
        # Sort by order
        docs = sorted(
            [doc for doc in docs if doc is not None],
            key=lambda d: d.metadata.get("order", 0),
        )
        return [
            {"sender": doc.metadata["sender"], "content": doc.page_content}
            for doc in docs
        ]

    def count(self) -> int:
        """Return the number of messages stored."""
        return len(self.vector_store.docstore._dict)

    def list_conversation_ids(self) -> list[str]:
        """List all unique conversation IDs stored."""
        ids = self.vector_store.docstore._dict.keys()
        return list(set(k.split("_")[0] for k in ids))

In [50]:
conv_id = uuid.uuid4()
store = FaissConversationStore(embedder)
messages = [
    {"sender": "human", "content": "Hello!"},
    {"sender": "ai", "content": "Hi, how can I help you?"},
    {"sender": "human", "content": "Tell me a joke."},
    {"sender": "ai", "content": "Why did the chicken cross the road?"},
]
store.store_conversation(str(conv_id), messages)
print(store.get_conversation(str(conv_id)))

[{'sender': 'human', 'content': 'Hello!'}, {'sender': 'ai', 'content': 'Hi, how can I help you?'}, {'sender': 'human', 'content': 'Tell me a joke.'}, {'sender': 'ai', 'content': 'Why did the chicken cross the road?'}]


In [52]:
# read a yaml file
import yaml

def read_yaml_file(file_path):
    """
    Read a YAML file and return its contents.
    """
    with open(file_path, "r") as file:
        data = yaml.safe_load(file)
    return data
yaml_file_path = "server/config.yaml"
yaml_data = read_yaml_file(yaml_file_path)
yaml_data

{'db_index_name': 'test_index', 'history_index_name': 'history_index'}